In [1]:
import pandas as pd
import geopandas
import folium
import numpy
from folium.features import GeoJsonTooltip

In [2]:
"""
This map will visualise covid-19 cases in US counties. 
Input data will be read from a CDC issued csv file (covid data) and a geojson file (US counties data).
"""

'\nThis map will visualise covid-19 cases in US counties. \nInput data will be read from a CDC issued csv file (covid data) and a geojson file (US counties data).\n'

In [3]:
geojson = geopandas.read_file("files/georef-united-states-of-america-county.geojson")
geojson = geojson[["coty_code", "geometry"]]
geojson

,coty_code,geometry
0,01119,"POLYGON ((-87.85342 32.53209, -87.85961 32.532..."
1,02230,"POLYGON ((-135.03053 59.34635, -134.98974 59.3..."
2,05011,"POLYGON ((-92.35028 33.29884, -92.35226 33.293..."
3,05069,"POLYGON ((-92.03025 34.48954, -92.08334 34.490..."
4,05085,"POLYGON ((-92.12154 35.01220, -92.07081 35.011..."
...,...,...
3229,55033,"POLYGON ((-92.13520 44.68437, -92.02842 44.683..."
3230,55057,"POLYGON ((-89.78581 43.64105, -89.78490 43.641..."
3231,72011,"POLYGON ((-67.08200 18.25495, -67.07891 18.258..."
3232,72043,"POLYGON ((-66.33124 18.01589, -66.32574 18.018..."


In [4]:
covid_df = pd.read_csv("files/United_States_COVID-19_County_Level_of_Community_Transmission_as_Originally_Posted.csv")
covid_df

,state_name,county_name,fips_code,report_date,cases_per_100K_7_day_count_change,percent_test_results_reported_positive_last_7_days,community_transmission_level
0,Alabama,Barbour County,1005,2021/11/10,60.760,1.50,substantial
1,Alabama,Chilton County,1021,2021/11/10,114.790,7.51,high
2,Alabama,Choctaw County,1023,2021/11/10,suppressed,3.33,moderate
3,Alabama,Clay County,1027,2021/11/10,83.110,2.94,substantial
4,Alabama,Cleburne County,1029,2021/11/10,134.140,12.00,high
...,...,...,...,...,...,...,...
6437,Texas,Kinney County,48271,2021/11/11,0.000,9.09,substantial
6438,Virginia,Fairfax City,51600,2021/11/11,516.260,NaN,high
6439,Washington,Spokane County,53063,2021/11/11,256.890,17.29,high
6440,Wisconsin,Forest County,55041,2021/11/11,277.650,8.77,high


In [5]:
covid_df["fips_code"] = covid_df["fips_code"].map(str) #changes column data to a string
covid_df["fips_code"] = covid_df["fips_code"].str.zfill(5) #filling the leading zero
covid_df = covid_df[(covid_df["report_date"] == "2021/11/11")]

In [6]:
covid_df['new_cases_7days'] = covid_df['cases_per_100K_7_day_count_change'].str.replace(',', '')
covid_df['new_cases_7days'] = covid_df['new_cases_7days'].replace({'suppressed': numpy.nan}).astype(float)
covid_df['pct_positive_7days']=covid_df['percent_test_results_reported_positive_last_7_days']/100
covid_df.drop(['cases_per_100K_7_day_count_change', 'percent_test_results_reported_positive_last_7_days',
              'community_transmission_level'], axis=1, inplace=True)

C:\Users\huawei\AppData\Local\Temp\ipykernel_9324\3519516675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df['new_cases_7days'] = covid_df['cases_per_100K_7_day_count_change'].str.replace(',', '')
C:\Users\huawei\AppData\Local\Temp\ipykernel_9324\3519516675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_df['new_cases_7days'] = covid_df['new_cases_7days'].replace({'suppressed': numpy.nan}).astype(float)
C:\Users\huawei\AppData\Local\Temp\ipykernel_9324\3519516675.py:3: SettingWithCopy

In [7]:
covid_df.head()

,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
2819,Iowa,Story County,19169,2021/11/11,194.61,0.1150
3222,Alabama,Autauga County,01001,2021/11/11,127.08,0.0655
3223,Alabama,Bibb County,01007,2021/11/11,71.45,0.0444
3224,Alabama,Blount County,01009,2021/11/11,185.04,0.0689
3225,Alabama,Cherokee County,01019,2021/11/11,118.34,0.1205


In [8]:
df_final = geojson.merge(covid_df, left_on="coty_code", right_on="fips_code", how="outer") 
df_final = df_final[~df_final['geometry'].isna()]
df_final.head()

,coty_code,geometry,state_name,county_name,fips_code,report_date,new_cases_7days,pct_positive_7days
0,01119,"POLYGON ((-87.85342 32.53209, -87.85961 32.532...",Alabama,Sumter County,01119,2021/11/11,80.47,0.0065
1,02230,"POLYGON ((-135.03053 59.34635, -134.98974 59.3...",Alaska,Skagway Muny,02230,2021/11/11,0.00,NaN
2,05011,"POLYGON ((-92.35028 33.29884, -92.35226 33.293...",Arkansas,Bradley County,05011,2021/11/11,176.53,0.0552
3,05069,"POLYGON ((-92.03025 34.48954, -92.08334 34.490...",Arkansas,Jefferson County,05069,2021/11/11,41.90,0.0114
4,05085,"POLYGON ((-92.12154 35.01220, -92.07081 35.011...",Arkansas,Lonoke County,05085,2021/11/11,88.67,0.0506


In [9]:
us_map = folium.Map(location=[40, -96], zoom_start=4,tiles='openstreetmap')

In [10]:
custom_scale = (df_final['new_cases_7days'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()
folium.Choropleth(
            geo_data="files/georef-united-states-of-america-county.geojson",
            data=df_final,
            columns=['fips_code', 'new_cases_7days'],
            key_on='feature.properties.coty_code',
            threshold_scale=custom_scale,
            fill_color='YlOrRd',
            nan_fill_color="White",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='New Cases Past 7 Days (Per 100K Population) ',
            highlight=True,
            line_color='black').add_to(us_map) 

In [11]:
folium.features.GeoJson(
                    data=df_final,
                    name='New Cases Past 7 days (Per 100K Population)',
                    smooth_factor=2,
                    style_function=lambda x: {'color':'black','fillColor':'transparent','weight':0.5},
                    tooltip=folium.features.GeoJsonTooltip(
                        fields=['report_date',
                                'county_name',
                                'state_name',
                                'new_cases_7days',
                                'pct_positive_7days'
                               ],
                        aliases=["Report Date:",
                                 "County Name:",
                                 "State Name:",
                                 "New Cases Past 7 days<br>(Per 100K Population):",
                                 "Percent of Positive Cases<br>Past 7days:"
                                ], 
                        localize=True,
                        sticky=False,
                        labels=True,
                        style="""
                            background-color: #F0EFEF;
                            border: 2px solid black;
                            border-radius: 3px;
                            box-shadow: 3px;
                        """,
                        max_width=800,),
                            highlight_function=lambda x: {'weight':3,'fillColor':'grey'},
                        ).add_to(us_map)   

In [12]:
# us_map.save("Choropleth Map 1 - Covid US Geopandas.html") #to save as html (~30mb)
# us_map #to create a map in notebook (html file too big for github)